In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import preprocessing
from itertools import combinations
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import warnings
from sklearn.model_selection import GridSearchCV
warnings.filterwarnings("ignore")
import xgboost as xgb

In [2]:
df_train = pd.read_csv('train-2.csv')
df_test = pd.read_csv("test-2.csv")

In [3]:
df_train.head(2)

,Id,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,TARGET
0,57094,1.0,3,0,2016-05-19 18:06:27,A FABULOUS 3BR IN MIDTOWN WEST! PERFECT APAR...,HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,"['Laundry In Unit', 'No Fee', 'Elevator']",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,W 50 & AVE 10,medium
1,33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,Renovated Kitchen and Bathroom!,55 River Drive South,"['Dogs Allowed', 'Cats Allowed', 'No Fee']",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,55 River Drive South,medium


In [4]:
df_test.head(2)

,Id,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address
0,19477,2.0,3,87c4e08083ac83618dd9fdbf849331fe,2016-06-21 03:21:59,Rent listed is net effective after one month f...,E 77 St.,"['Pre-War', 'Laundry in Unit', 'Dishwasher', '...",40.7716,7189305,-73.9544,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7189305_f47ed5c...,4675,339 E 77 St.
1,29937,1.0,2,850c3d4412aebb3335273ed95f3e4bd4,2016-06-11 03:47:06,A FABULOUS 2BR IN THE UPPER EAST SIDE! PERFECT...,E 112 Street,[],40.7944,7140670,-73.9401,3e9f0fa34d67e5d61aae56776ecb8cee,['https://photos.renthop.com/2/7140670_6c5824f...,2100,252 E 112 Street


In [5]:
le = preprocessing.LabelEncoder()
le.fit(list(df_train['TARGET']))
le.transform(list(df_train['TARGET']))

array([2, 2, 1, ..., 1, 2, 2], dtype=int64)

In [6]:
df_train['target'] = pd.DataFrame(le.transform(list(df_train['TARGET'])))
df_train = df_train.drop('TARGET', axis=1)

In [7]:
df_train['target'].value_counts()

1    23999
2     7860
0     2687
Name: target, dtype: int64

Нужно бороться с дисбалансом классов

In [8]:
class0 = pd.concat([df_train[df_train['target'] == 0] , 
                    df_train[df_train['target'] == 0], 
                    df_train[df_train['target'] == 0], 
                    df_train[df_train['target'] == 0]])

In [9]:
class1 = df_train[df_train['target'] == 1].sample(11798)

In [10]:
class2 = df_train[df_train['target'] == 2].sample(4140)

In [11]:
df_train_sample = pd.concat([class1, class2, class0, df_train[df_train['target'] == 2]]).sample(frac=1)

In [12]:
df_train_sample = df_train_sample.reset_index(drop=True)

In [13]:
df_train_sample['target'].value_counts()

2    12000
1    11798
0    10748
Name: target, dtype: int64

In [14]:
X_train = pd.DataFrame(df_train_sample.drop('target', axis=1))

In [15]:
df = pd.concat([X_train, df_test])
df = df.reset_index(drop=True)
df.head(3)

,Id,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address
0,53615,1.0,2,a01c99eb2cfdde327e1691e17d6696ba,2016-05-20 05:40:09,AMAZING and SPACIOUS 2 bedroom convertible. BE...,East 29th Street,"['Balcony', 'Doorman', 'Elevator', 'Garden/Pat...",40.7412,7043777,-73.9772,1fb46c4a72bcf764ac35fc23f394760d,['https://photos.renthop.com/2/7043777_5ec00b8...,2965,340 East 29th Street
1,61143,2.0,4,d876f52470ecafaf62629e75349d439d,2016-05-05 05:50:19,LOW FEE!!!!! luxury HUGE! 4 bedroom conv. w/ l...,East 57th Street,"['Dining Room', 'Doorman', 'Elevator', 'Dishwa...",40.7605,6969940,-73.9678,1fb46c4a72bcf764ac35fc23f394760d,['https://photos.renthop.com/2/6969940_1167992...,5000,157 East 57th Street
2,66599,1.0,1,2c844ae451aa68c1358c2cc4d78cff5e,2016-05-28 02:58:09,NO BROKER FEETHIS GORGEOUS 1 BEDROOM HAS A MA...,Main Street,"['Doorman', 'Elevator', 'Fitness Center', 'Gar...",40.7690,7078711,-73.9436,8ce0dca957ae00f6870a164ead5081f8,['https://photos.renthop.com/2/7078711_154d4f9...,2300,888 Main Street


In [16]:
df = df.drop(['description', 'street_address', 'listing_id'], axis=1)

In [17]:
def encoder(df, column):
    le = preprocessing.LabelEncoder()
    le.fit(list(df[column]))
    le.transform(list(df[column]))
    return pd.DataFrame(le.transform(list(df[column])))

In [18]:
df['build_id'] = encoder(df, 'building_id')
df = df.drop('building_id', axis=1)

In [19]:
df['dis_address'] = encoder(df, 'display_address')
df = df.drop('display_address', axis=1)

In [20]:
df['manag_id'] = encoder(df, 'manager_id')
df = df.drop('manager_id', axis=1)

In [21]:
df['created'] = pd.to_datetime(df['created'], infer_datetime_format=True)

In [22]:
month = []
for i in tqdm(range(len(df))):
    month.append(df['created'].dt.month[i])

In [23]:
day = []
for i in tqdm(range(len(df))):
    day.append(df['created'].dt.day[i])

In [24]:
hour = []
for i in tqdm(range(len(df))):
    hour.append(df['created'].dt.hour[i])

In [25]:
df['month'] = pd.DataFrame(month)
df['day'] = pd.DataFrame(day)
df['hour'] = pd.DataFrame(hour)

In [26]:
df = df.drop('created', axis=1)

Заменяю листы с ссылками фоток на количество ссылок в листах

In [27]:
count_photos = []
for i in range(len(df)):
    count_photos.append(len(df['photos'][i]))

In [28]:
df = df.drop('photos', axis=1)
df['count_photos'] = pd.DataFrame(count_photos, columns=['count_photos'])

Заменяю колонку "features" на количество фичей в конкретном доме

In [29]:
features = []
for j in range(len(df)):
    spisok = []
    if len(df['features'][j]) > 2:
        for i in range(len((df['features'][j][2:-2]).split("'"))):
            if df['features'][j][2:-2].split("'")[i][0] in ('ABCDEFGHIJKLMNOPQRSTUVWXYZ' or 'abcdefghijklmnopqrstuvwxyz'):
                spisok.append(df['features'][j][2:-2].split("'")[i])
        features.append(spisok)
    else:
        features.append([])

In [30]:
count_features = []
for i in range(len(features)):
    count_features.append(len(features[i]))
df['count_features'] = pd.DataFrame(count_features)
df = df.drop('features', axis=1)

Добавляю новые фичи

In [31]:
df['price2'] = df['price'] * df['price']
df['price_photos'] = df['price'] * df['count_photos']
df['price_features'] = df['price'] * df['count_features']

In [32]:
X_train = df[:34546]
X_test = df[34546:]

In [33]:
X_test = X_test.reset_index(drop=True)

In [34]:
y_train = df_train_sample['target']

In [35]:
model = xgb.XGBClassifier(n_estimators=1000, max_depth=4, objective='multi:softmax')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

[14:13:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [36]:
pd.DataFrame(y_pred).value_counts()

1    8719
2    4811
0    1276
dtype: int64

In [37]:
submission = pd.DataFrame()
submission['Id'] = X_test['Id']
submission['TARGET'] = pd.DataFrame(le.inverse_transform(list(y_pred)))
submission.to_csv('submission.csv', index=False)